In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.layers import concatenate, Dropout, GlobalAveragePooling2D
from tensorflow.keras.layers import BatchNormalization, Activation, AveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# seed
import os
seed = 123
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.set_random_seed(seed)

In [2]:
train = np.load('data/train.npy', allow_pickle = 'True')
test = np.load('data/test.npy', allow_pickle = 'True')

In [3]:
x = train[:,2:]
x = np.reshape(x, (-1, 28, 28, 1))
x = x/255

y_letter = train[:,1]
y_letter = np.reshape(y_letter, (-1, 1))
en = OneHotEncoder()
y_letter = en.fit_transform(y_letter).toarray()

y = train[:,0]
y = np.reshape(y, (-1, 1))
en = OneHotEncoder()
y = en.fit_transform(y).toarray()

print(x.shape)
print(y_letter.shape)
print(y.shape)

(2048, 28, 28, 1)
(2048, 26)
(2048, 10)


In [4]:
valid_size = 48
valid_x = x[-valid_size:]
x = x[:-48]

valid_y_letter = y_letter[-valid_size:]
y_letter = y_letter[:-48]

valid_y = y[-valid_size:]
y = y[:-48]

print(x.shape)
print(valid_x.shape)
print(y_letter.shape)
print(valid_y_letter.shape)
print(y.shape)
print(valid_y.shape)

(2000, 28, 28, 1)
(48, 28, 28, 1)
(2000, 26)
(48, 26)
(2000, 10)
(48, 10)


In [5]:
image_generator = ImageDataGenerator(width_shift_range=0.1,
                                     height_shift_range=0.1, 
                                     zoom_range=[0.8,1.2],
                                     shear_range=10)

In [6]:
x_total = x.copy()
def augment(x):
    aug_list = []
    for i in range(x.shape[0]):
        num_aug = 0
        tmp = x[i]
        tmp = tmp.reshape((1,) + tmp.shape)
        for x_aug in image_generator.flow(tmp, batch_size = 1) :
            if num_aug >= 1:
                break
            aug_list.append(x_aug[0])
            num_aug += 1
    aug_list = np.array(aug_list)
    return aug_list

n = 2
for i in range(n):
    arr = augment(x)
    x_total = np.concatenate((x_total, arr), axis=0)
    if i > n:
        break

print(x_total.shape)

(6000, 28, 28, 1)


In [7]:
y_total = y.copy()
for i in range(n):
    arr = y.copy()
    y_total = np.concatenate((y_total, arr), axis=0)

print(y_total.shape)

(6000, 10)


In [8]:
y_letter_total = y_letter.copy()
for i in range(n):
    arr = y_letter.copy()
    y_letter_total = np.concatenate((y_letter_total, arr), axis=0)
    
print(y_letter_total.shape)

(6000, 26)


In [9]:
x_train, x_val, y_train, y_val = train_test_split(x_total, y_total, test_size=0.2, shuffle=True)#, stratify=y_total)
y_letter_train = y_letter_total[:x_train.shape[0],:]
y_letter_val = y_letter_total[x_train.shape[0]:,:]

print(x_train.shape)
print(x_val.shape)
print(y_train.shape)
print(y_val.shape)
print(y_letter_train.shape)
print(y_letter_val.shape)

(4800, 28, 28, 1)
(1200, 28, 28, 1)
(4800, 10)
(1200, 10)
(4800, 26)
(1200, 26)


In [10]:
input1 = Input(shape=(28,28,1))
x1 = Conv2D(64, (3,3), activation='relu', padding='same')(input1)
x1 = Conv2D(64, (3,3), activation='relu', padding='same')(x1)
#x1 = Dropout(0.3)(x1)
x1 = MaxPooling2D((2,2))(x1)
x1 = Conv2D(64, (2,2), activation='relu', padding='same')(x1)
x1 = Conv2D(64, (2,2), activation='relu', padding='same')(x1)
#x1 = Dropout(0.3)(x1)
x1 = MaxPooling2D((2,2))(x1)
x1 = Conv2D(128, (2,2), activation='relu', padding='same')(x1)
x1 = Conv2D(64, (2,2), activation='relu', padding='same')(x1)
#x1 = Dropout(0.3)(x1)
x1 = GlobalAveragePooling2D()(x1)
outputs = Dense(26, activation='softmax')(x1)

model = Model(inputs = input1, outputs = outputs)

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 28, 28, 64)        640       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 64)        36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 64)        16448     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 14, 14, 64)        16448     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 64)          0         
__________

In [11]:
model.compile(optimizer = 'adam', metrics = ['accuracy'], loss = 'categorical_crossentropy')

#es = EarlyStopping(monitor='val_loss', patience=20, mode='min', verbose=1)
cp = ModelCheckpoint('./models/{epoch:02d}-{val_acc:.4f}.h5', monitor='val_loss',
                     save_best_only=True, mode='min')

history = model.fit(x_train, y_letter_train,
                    validation_data=(x_val, y_letter_val), 
                    batch_size=64, epochs=100, verbose=1, callbacks = [cp])

Train on 4800 samples, validate on 1200 samples
Epoch 1/100
4800/4800 [==============================] - 5s 1ms/step - loss: 3.2576 - acc: 0.0331 - val_loss: 3.2551 - val_acc: 0.0442
Epoch 2/100
4800/4800 [==============================] - 2s 365us/step - loss: 3.2554 - acc: 0.0402 - val_loss: 3.2542 - val_acc: 0.0458
Epoch 3/100
4800/4800 [==============================] - 2s 376us/step - loss: 3.2544 - acc: 0.0435 - val_loss: 3.2540 - val_acc: 0.0458
Epoch 4/100
4800/4800 [==============================] - 2s 381us/step - loss: 3.2536 - acc: 0.0435 - val_loss: 3.2536 - val_acc: 0.0458
Epoch 5/100
4800/4800 [==============================] - 2s 357us/step - loss: 3.2535 - acc: 0.0396 - val_loss: 3.2536 - val_acc: 0.0458
Epoch 6/100
4800/4800 [==============================] - 2s 364us/step - loss: 3.2539 - acc: 0.0427 - val_loss: 3.2539 - val_acc: 0.0442
Epoch 7/100
4800/4800 [==============================] - 2s 378us/step - loss: 3.2532 - acc: 0.0423 - val_loss: 3.2536 - val_acc: 0.

4800/4800 [==============================] - 2s 378us/step - loss: 2.7017 - acc: 0.2217 - val_loss: 3.7879 - val_acc: 0.0458
Epoch 61/100
4800/4800 [==============================] - 2s 371us/step - loss: 2.5364 - acc: 0.2646 - val_loss: 3.8993 - val_acc: 0.0325
Epoch 62/100
4800/4800 [==============================] - 2s 374us/step - loss: 2.3871 - acc: 0.3015 - val_loss: 4.1148 - val_acc: 0.0367
Epoch 63/100
4800/4800 [==============================] - 2s 371us/step - loss: 2.2013 - acc: 0.3598 - val_loss: 4.3100 - val_acc: 0.0367
Epoch 64/100
4800/4800 [==============================] - 2s 374us/step - loss: 2.0028 - acc: 0.4217 - val_loss: 4.5673 - val_acc: 0.0333
Epoch 65/100
4800/4800 [==============================] - 2s 381us/step - loss: 1.8105 - acc: 0.4658 - val_loss: 4.9356 - val_acc: 0.0350
Epoch 66/100
4800/4800 [==============================] - 2s 371us/step - loss: 1.5981 - acc: 0.5354 - val_loss: 5.2252 - val_acc: 0.0425
Epoch 67/100
4800/4800 [=======================

KeyboardInterrupt: 

In [ ]:
#from tensorflow.keras.models import load_model
#best_model = load_model('models/DenseNet121.h5')

In [ ]:
#evaluate = best_model.evaluate(valid_x, valid_y_letter)
#evaluate

In [ ]:
#submission = pd.read_csv('data/val.csv')
#submission['digit'] = np.argmax(best_model.predict([x_test, x_letter_test]), axis=1)
#submission.to_csv('data/val(CNN).csv', index=False)